# Content and Objective

+ Show calculations of Levinson-Durbin
+ Method: Get random ar signal and apply equations of LD and YW, respectively

In [ ]:
# importing
import numpy as np
import scipy.signal
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib

# showing figures inline
%matplotlib inline

In [ ]:
# plotting options
font = {'size': 30}
plt.rc('font', **font)
plt.rc('text', usetex=True)

matplotlib.rc('figure', figsize=(30, 8))

# Get Parameter by Applying YW

In [ ]:
########################
# acf estimator
########################
def est_acf(y, est_type):
    """
    estimates acf given a number of observation
    
    Remark: signal is assumed to be starting from 0 to length(y)-1
    
    IN: observations y, est_type (biased / unbiased)
    OUT: estimated acf, centered around 0
    """

    N = np.size(y)
    r = np.zeros_like(y)

    # loop lags of acf
    for k in np.arange(0, N):

        temp = np.sum(y[k:N] * (y[0:(N - k)]))

        # type of estimator
        if est_type == 'biased':
            r[k] = temp / N
        elif est_type == 'unbiased':
            r[k] = temp / (N - k)

    # find values for negative indices
    r_reverse = np.conjugate(r[::-1])

    return np.append(r_reverse[0:len(r) - 1], r)


##########################
# YW parameters
##########################


def find_parameters_yulewalker(y, order, s2):
    """
    estimates a_v parameters of filter using the Yule-Walker method
                
    IN: samples y, YW oder q
    OUT: parameters a_v
    """

    N = len(y)

    #r = est_acf(y, 'biased')
    r = np.correlate(y, y, 'full')
    r /= N

    # get matrix R for Yule-Walker
    # note that R is not the autocorrelation matrix, but R = (ACF matrix)^*
    R = np.zeros([order + 1, order + 1], dtype=float)
    for p in range(0, order + 1):
        R[:, p] = r[N - 1 - p:N - 1 - p + order + 1]

    # find and solve linear equation system for the coefficients
    b = np.matrix(np.append(s2, np.zeros(order))).T
    theta = np.linalg.solve(R, b)

    theta = np.array(theta / theta[0])

    return theta.T

# Construct Signal

### Parameters

In [ ]:
# parameters: noise variance
sigma_x_2 = 1 + np.random.rand()
print(f'Noise variance: \t sigma2 = {sigma_x_2}')

# parameters: ar parameters
q = 2
a_1_q = np.random.rand(q)
a_1_q = np.array([.5, .25])
print(f'Feedback parameters: \t a = {a_1_q}')


### Sample noise and apply to signal

In [ ]:
##########################
# Get AR Result
##########################


def get_ar_signal(sigma_x_2, N_length):
    """
    estimates a_v parameters of filter using the Yule-Walker method
                
    IN: samples y, YW oder q
    OUT: parameters a_v
    """

    # init input noise and output as first input value
    x = np.sqrt(sigma_x_2) * np.random.randn(N_length)
    y = np.zeros_like(x)

    # loop for times
    for n, value in enumerate(x):
        ar = 0.0
        for _q in range(1, np.min((n, q)) + 1):
            ar += a_1_q[_q - 1] * y[n - _q]

        # apply
        y[n] = value - ar

    if 0:
        print(f'Input noise: \t x = {x}')
        print(f'Output signal: \t y = {y}')

    return y

# Apply Levinson-Durbin

+ Init: 
$$\theta_1=-\frac{\rho_1}{\rho_0}, \quad k_1=\theta_1, \quad \sigma_1^2=\rho_0-\frac{|\rho_1|^2}{\rho_0}$$

+ Loop q = 1 : max:
$$ k_{q+1}=-\frac{ \rho_{q+1}+\mathbf{r}_{q,\leftarrow}\mathrm{\theta}_q }{\sigma_q^2}$$
$$\sigma_{q+1}^2 =\sigma_q^2 ( 1-|k_{q+1}|^2)$$
$$\mathbf{\theta}_{q+1} = \begin{pmatrix} \mathbf{\theta}_q \\ 0 \end{pmatrix} + k_{q+1}\begin{pmatrix} \mathbf{\theta}_{q, \leftarrow} \\ 1 \end{pmatrix}$$

In [ ]:
# max value of q
q_max = 15

# sequence length
N_len = int(1e3)

# get ar signal
y = get_ar_signal(sigma_x_2, N_len)

# estimate acf and get indices/shifts >= 0 only
r = est_acf(y, 'biased')
r_pos = r[(len(r) - 1) // 2:]

# init LD
theta = [-r_pos[1] / r_pos[0]]
k = theta
sigma2 = r_pos[0] - np.abs(r_pos[1])**2 / r_pos[0]

# would you like to see init values?
if 0:
    print('Results of initialization:')
    print('--------------------------')
    print(f'theta = \t {theta} ')
    print(f'k = \t\t {k} ')
    print(f'sigma2 = \t {sigma2}\n')

# loop for orders
for _q in range(1, q_max + 1):

    # slice acf values and determine inverted values
    r_q = r_pos[1:_q + 1]
    r_q_left = r_q[::-1]

    # determine new k, sigma2 and theta
    k = -(r_pos[_q + 1] + np.inner(r_q_left, theta)) / sigma2

    sigma2 = sigma2 * (1 - np.abs(k)**2)

    theta = np.append(theta, 0) + k * np.append(theta[::-1], 1)

    # like to see intermediate results?
    if 1:
        print(f'q = {_q}')
        print('---------')
        print(f'Current k: \t\t{k}')
        print(f'Current sigma2: \t {sigma2}')
        print(f'Current theta: \t\t {theta}')
        print(
            f'YW: theta: \t\t {find_parameters_yulewalker(y, _q+1, sigma2 )[0][1:]} \n'
        )

# like to see final results?
if 1:
    print('Final results:')
    print('--------------')
    print(f'k: \t\t{k}')
    print(f'sigma2: \t {sigma2}\n')
    print(f'theta: \t\t {theta}\n')
    print(
        f'YW: theta: \t {find_parameters_yulewalker(y, q_max+1, sigma2 )[0][1:]} \n'
    )


# Redefine LD as Function to Time it

In [ ]:
##########################
# define LD algorithm
##########################


def find_parameters_levinsondurbin_based_on_y(y, q_max):
    """
    estimates a_v parameters of filter using the Levinson-Durbin method
                
    IN: samples y, order q_max
    OUT: parameters a_v
    """

    r = est_acf(y, 'biased')
    r_pos = r[(len(r) - 1) // 2:]

    # init
    theta = [-r_pos[1] / r_pos[0]]
    k = theta
    sigma2 = r_pos[0] - np.abs(r_pos[1])**2 / r_pos[0]

    if 0:
        print('Results of initialization:')
        print('--------------------------')
        print(f'theta = \t {theta} ')
        print(f'k = \t\t {k} ')
        print(f'sigma2 = \t {sigma2}\n')

    for _q in range(1, q_max + 1):

        r_q = r_pos[1:_q + 1]
        r_q_left = r_q[::-1]

        k = -(r_pos[_q + 1] + np.inner(r_q_left, theta)) / sigma2

        sigma2 = sigma2 * (1 - np.abs(k)**2)

        theta = np.append(theta, 0) + k * np.append(theta[::-1], 1)

    return theta


def find_parameters_levinsondurbin_based_on_r(r_pos, q_max):
    """
    estimates a_v parameters of filter using the Levinson-Durbin method
                
    IN: samples y, order q_max
    OUT: parameters a_v
    """

    # init
    theta = [-r_pos[1] / r_pos[0]]
    k = theta
    sigma2 = r_pos[0] - np.abs(r_pos[1])**2 / r_pos[0]

    if 0:
        print('Results of initialization:')
        print('--------------------------')
        print(f'theta = \t {theta} ')
        print(f'k = \t\t {k} ')
        print(f'sigma2 = \t {sigma2}\n')

    for _q in range(1, q_max + 1):

        r_q = r_pos[1:_q + 1]
        r_q_left = r_q[::-1]

        k = -(r_pos[_q + 1] + np.inner(r_q_left, theta)) / sigma2

        sigma2 = sigma2 * (1 - np.abs(k)**2)

        theta = np.append(theta, 0) + k * np.append(theta[::-1], 1)

    return theta


def find_parameters_yulewalker_based_on_r(r, order, s2):
    """
    estimates a_v parameters of filter using the Yule-Walker method
                
    IN: samples y, YW oder q
    OUT: parameters a_v
    """

    N = int((len(r) + 1) / 2)

    # get matrix R for Yule-Walker
    # note that R is not the autocorrelation matrix, but R = (ACF matrix)^*
    R = np.zeros([order + 1, order + 1], dtype=float)
    for p in range(0, order + 1):
        R[:, p] = r[N - 1 - p:N - 1 - p + order + 1]

    # find and solve linear equation system for the coefficients
    b = np.matrix(np.append(s2, np.zeros(order))).T
    theta = np.linalg.solve(R, b)

    theta = np.array(theta / theta[0])

    return theta.T

### time it

In [ ]:
import time

# max value of q
q_max = 100

# sequence length
N_len = int(1e4)

# get ar signal
y = get_ar_signal(sigma_x_2, N_len)

r = est_acf(y, 'biased')
r_pos = r[(len(r) - 1) // 2:]

# do LD
start = time.time()
#t = find_parameters_levinsondurbin_based_on_y( y, q_max)
t = find_parameters_levinsondurbin_based_on_r(r_pos, q_max)
elapsed = time.time() - start

print(f'LD required: {elapsed}')

# do YW
start = time.time()
for _q in range(q_max + 1):
    #t = find_parameters_yulewalker( y, q_max,sigma2)
    t = find_parameters_yulewalker_based_on_r(r, q_max, sigma2)
elapsed = time.time() - start

print(f'YW required: {elapsed}')
